<a href="https://colab.research.google.com/github/ling-lyanna-zhang/reserve-neural-policy/blob/main/try_gurobi_reserve_39bus_(step1)benchmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# in case gpu is used
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [3]:
import os, time
import cvxpy as cp
import random
from numpy import savetxt
import argparse
import json
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np

from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
from torch.autograd import Variable

root_path = './gdrive/MyDrive/Inbox/rs_neural_policy/'


In [4]:
# nn.init.trunc_normal__(tensor: Tensor, 
#                        mean: float = 0., 
#                        std: float = 1., 
#                        a: float = -2., 
#                        b: float = 2.)

In [5]:
# Install MOSEK if not already installed
!pip install mosek

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
print(cp.installed_solvers())

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'GUROBI', 'MOSEK', 'OSQP', 'SCIPY', 'SCS']


# helper functions, can be folded

## utils.py

In [7]:
def create_dir(PATH):
    isExist = os.path.exists(PATH)

    if not isExist:
        # Create a new directory if it does not exist 
        os.makedirs(PATH)
        print("The new directory is created!")


# Evaluate using vector distance
def measure_relative_distance(v1, v2):
    '''
        Note that v1 is the benchmark.
        Norm is calculated along dimension/axis 1
        and average is calculated along dimension/axis 0
        Also return the distance vetor.
    '''
    if len(v1.shape)==1 and len(v2.shape)==1:
        distance = np.abs(v1-v2)/np.abs(v1)

    if len(v1.shape)==2 and len(v2.shape)==2:
        distance = np.linalg.norm(v1-v2, axis=1)/np.linalg.norm(v1, axis=1)

    return distance, np.mean(distance)

## paths.py

In [8]:
def get_paths():
    params_path = root_path+'39bus/params/'
    data_path = root_path+'39bus/data/'
    model_path = root_path+'39bus/saved_models/'
    saved_path = root_path+'39bus/predictions/'


    isExist = os.path.exists(data_path)
    if not isExist:
        # Create a new directory if it does not exist 
        os.makedirs(data_path)

    isExist = os.path.exists(model_path)
    if not isExist:
        # Create a new directory if it does not exist 
        os.makedirs(model_path)

    
    isExist = os.path.exists(saved_path)
    if not isExist:
        # Create a new directory if it does not exist 
        os.makedirs(saved_path)

    return params_path, data_path, saved_path, model_path

In [9]:
def load_cost_coeff(params_path):    
    quad_cost_coeff = np.load(params_path+'quad_cost_coeff.npy')
    linear_cost_coeff = np.load(params_path+'linear_cost_coeff.npy')
    da_cost_coeff = np.load(params_path+'da_cost_coeff.npy')

    return quad_cost_coeff, linear_cost_coeff, da_cost_coeff


## system.py

In [10]:
def identify_unique_lines(connections):
    all_lines = {}
    count = 0
    for line in connections:
        all_lines[count] = line
        count+=1

    # This code snippet only finds out the repeated lines with exactly the same order of nodes,
    # but not deal with that [i,j] and [j,i] are also repeated lines
    # By checking connections, there is no repeated lines like [i,j] and [j,i]
    unique_lines = {}
    for k, val in all_lines.items():
        if val not in unique_lines.values():
            unique_lines[k]=val
    print('unique_lines length:', len(unique_lines))

    repeated_lines = [[42, 49],[49, 54],[56, 59],[49, 66],[77, 80],[89, 90],[89, 92]]
    # For example, [42, 49] appears twice
    set1 = {}
    set2 = {}
    for k, val in all_lines.items():
        if val in repeated_lines and k in unique_lines:
            set1[val[0], val[1]] = k # Record the repeated lines when they first appear
        if val in repeated_lines and k not in unique_lines:
            set2[val[0], val[1]] = k # Record the repeated lines when they appear more than once

    # print('set1:', len(set1))
    # print('set2:', len(set2))

    return unique_lines, set1, set2


def get_Y(N, B):
    # N = num_buses
    Y = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if i==j : 
                Y[i,j] = sum(B[i,:])
            else: 
                Y[i,j] = -B[i,j]

    return Y[:,1:]


def get_A(N, L, B, connections):
    # N = num_buses
    # L = num_lines 
    A = np.zeros((L, N))

    for i, line in enumerate(connections):
        row = line[0]-1
        col = line[1]-1
        A[i, row] = B[row,col]
        A[i, col] = -B[row,col]

    return A[:,1:]


# Import 39bus
def import_39bus(params_path):
    bus_data_fname = '39bus_BusData.csv'
    gen_data_fname = '39bus_GenData.csv'
    branch_data_fname = '39bus_BranchData.csv'
    cost_data_fname = '39bus_CostData.csv'

    bus_data_df = pd.read_csv(params_path+bus_data_fname,header=None)
    gen_data_df = pd.read_csv(params_path+gen_data_fname,header=None)
    branch_data_df = pd.read_csv(params_path+branch_data_fname,header=None)
    cost_data_df = pd.read_csv(params_path+cost_data_fname,header=None)


    num_buses = bus_data_df.shape[0]
    num_lines = branch_data_df.shape[0]
    num_gens = gen_data_df.shape[0]


    bus_data = bus_data_df.to_numpy()
    gen_data = gen_data_df.to_numpy()
    branch_data = branch_data_df.to_numpy()
    cost_data = cost_data_df.to_numpy()

    x = branch_data[:,3]
    print('max of x:', max(x))
    print('min of x:', min(x))


    b = 1/x
    Z0 = 10
    b = b/Z0
    print('max of b:', max(b))
    print('min of b:', min(b))

    connections = []
    branches = branch_data[:,:2]
    for i in range(branches.shape[0]):
            connections.append([int(branches[i,0]),int(branches[i,1])])
    print('len of connections:', len(connections))

    unique_lines, set1, set2 = identify_unique_lines(connections)

    B = np.zeros((num_buses, num_buses))
    for k, line in unique_lines.items():
        row = line[0]-1
        col = line[1]-1
        B[row, col] = b[k]
        B[col, row] = b[k]

    PD = bus_data[:,2]/20
    print('Total PD:', sum(PD))

    return num_buses, num_lines, B, connections, PD

### import B, F and cost

In [11]:
params_path, data_path, saved_path, model_path = get_paths()

num_buses, num_lines, B, connections, PD_data = import_39bus(params_path)

N = num_buses
L = num_lines 

Yrr = get_Y(num_buses, B)
Arr = get_A(num_buses, num_lines, B, connections)

# # Define feasibility set for gauge mapping
# G = np.block([
#               [Arr],
#               [-Arr],
#               [np.eye(N-1)],
#               [-np.eye(N-1)]
# ])
# print('G shape:', G.shape)

# Load cost coefficients
quad_cost_coeff, linear_cost_coeff, da_cost_coeff = load_cost_coeff(params_path) 

quad_cost_Coeff = np.diag(quad_cost_coeff)
linear_cost_Coeff = linear_cost_coeff.reshape(-1, 1)
da_cost_Coeff = da_cost_coeff.reshape(-1,1)

print('quad_cost_Coeff shape:', quad_cost_Coeff.shape)
print('linear_cost_Coeff shape:', linear_cost_Coeff.shape)
print('da_cost_Coeff shape:', da_cost_Coeff.shape)

c = da_cost_Coeff
q = linear_cost_Coeff

max of x: 0.062
min of x: 0.003
max of b: 33.33333333333333
min of b: 1.6129032258064515
len of connections: 46
unique_lines length: 46
Total PD: 156.35575
quad_cost_Coeff shape: (39, 39)
linear_cost_Coeff shape: (39, 1)
da_cost_Coeff shape: (39, 1)


### set Pmax/Pmin, Fmax and std

In [12]:
DayAheadReserveCostFactor = 1.1
RecourseReserveCostFactor = 5.
Pmax = 30.
Pmin = 0.
Fmax = 15.

input_std = 0.1
sce_std = 0.05

## dataloader.py

In [13]:
def for_sinlge_instance(forecast, x):

    p0 = x[:N,:]
    r_up = x[N:N+N,:]
    r_down = x[N+N:,:]

    theta = cp.Variable((N-1,1))
    y = cp.Variable((N, 1))

    constraints_list = []

    PD = forecast.flatten()
    std_vec = PD*sce_std
    std_vec = np.where(std_vec<1e-5, 0.1, std_vec)

    rt_scenario = np.random.multivariate_normal(PD, np.diag(std_vec), 1)
    # generated scenario are arranged in rows, must be transposed
    rt_scenario = np.clip(rt_scenario, a_min=0., a_max=None).T
    assert rt_scenario.shape[0] == N
    
    net_d_omega = rt_scenario - p0

    constraints_list.append( y == Yrr@theta + net_d_omega )

    constraints_list.append( p0 + y <= Pmax*np.ones((N, 1)))
    constraints_list.append( p0 + y >= Pmin*np.ones((N, 1)))
    # constraints_list.append( y <= r_up)
    # constraints_list.append( y >= -r_down)
        
    constraints_list.append( Arr@theta <= Fmax*np.ones((num_lines, 1)))
    constraints_list.append( Arr@theta >= -Fmax*np.ones((num_lines, 1)))

    constraints_list.append( theta<=np.pi)
    constraints_list.append( theta>=-np.pi)

    recourse_cost = cp.abs(y).T@q +\
        cp.pos(y-r_up).T@q*RecourseReserveCostFactor +\
        cp.pos(y+r_down).T@q*RecourseReserveCostFactor

    Q_pred = recourse_cost

    prob = cp.Problem(cp.Minimize( Q_pred ), constraints_list)

    prob.solve(verbose = False, solver = cp.ECOS) ## Works!   

    return prob.value, y.value, theta.value, net_d_omega, prob.status

In [14]:
def solve_stochastic_dcopf(forecasts, x_vals):
    '''
    Given a single instance and the initial dispatch x, and evaluate the quality
    of x.
    This function can be applied to the decision obtained from any policy.
    '''
    num_points = forecasts.shape[0]

    Q_vals = []
    y_vals = []
    theta_vals = []
    net_d_vals = []
    for i in range(num_points):
        forecast = forecasts[i,:].reshape(-1,1)
        x = x_vals[i,:].reshape(-1,1)

        Q_omega, y_omega, theta_omega, net_d_omega, prob_status = for_sinlge_instance(forecast, x)

        if prob_status in ["infeasible", "unbounded"]:
            print("Problem is " + str(prob_status))
            Q_omega = 0.
            y_omega = np.zeros((N, 1))
            theta_omega = np.zeros((N-1, 1))

        Q_vals.append(Q_omega)
        y_vals.append(y_omega.flatten())
        theta_vals.append(theta_omega.flatten())
        net_d_vals.append(net_d_omega.flatten())

    Q_vals = np.array(Q_vals).reshape(-1,1)
    y_vals = np.array(y_vals)
    theta_vals = np.array(theta_vals)
    net_d_vals = np.array(net_d_vals)

    return Q_vals, y_vals, theta_vals, net_d_vals


## benchmarks.py

### solver

In [15]:
def solve_GAP(forecast, num_sce):
    '''
    Implemented by following the GAP model in [Kannan, Luedtke, Roald(2020)].
    I think it is kind of weird that y doesn't have an offset and the second-stage
    is totally determined by the uncertainties.
    So implement modified GAP instead.
    '''

    M = num_sce

    # c = da_cost_Coeff
    # q = linear_cost_Coeff
    B = -Yrr
    F = Arr
    rho_lb = 10.
    rho_ub = 10.
    gamma_lb = 10.
    gamma_ub = 10.
    ones = np.ones((N,1))

    Brr = -Yrr[1:,:]
    Brr_inv = np.linalg.inv(Brr)
    I = np.eye(N)
    R = I[1:,:]
    
    # first-stage decision variables
    x = cp.Variable((N, 1))
    r_up = cp.Variable((N, 1))
    r_down = cp.Variable((N, 1))
    alpha = cp.Variable((N, 1))

    # no second-stage decision variables
    # beta = cp.Variable((N,1))

    # theta = cp.Variable((N-1,M))

    constraints_list = []

    Q_vals = []

    # generate M uncertainty realizations
    PD = forecast.flatten()
    std_vec = PD*sce_std
    std_vec = np.where(std_vec<1e-5, 0.1, std_vec)

    scenarios = np.random.multivariate_normal(PD, np.diag(std_vec), M)
    scenarios = np.clip(scenarios, a_min=0., a_max=None).T
    assert scenarios.shape[0] == N

    for m in range(M):
        # for each scenario of noise
        omega =  scenarios[:, m:m+1] - forecast
        net_d = scenarios[:, m:m+1] - x

        # y_pred = (ones.T@omega)*alpha + beta
        y_pred = (ones.T@omega)*alpha
        theta_pred = Brr_inv@R@(net_d-y_pred)

        constraints_list.append( y_pred == Yrr@theta_pred + net_d )

        p_f_lb = rho_lb*cp.square( cp.norm( cp.pos(-Fmax*np.ones((num_lines, 1)) - F@theta_pred) ) )
        p_f_ub = rho_ub*cp.square( cp.norm(  cp.pos(F@theta_pred -Fmax*np.ones((num_lines, 1))) ) )

        p_theta_lb  = rho_lb*cp.square( cp.norm(  cp.pos(-np.pi*np.ones((N-1, 1)) - theta_pred) ) )
        p_theta_ub  = rho_ub*cp.square( cp.norm(  cp.pos(theta_pred - np.pi*np.ones((N-1, 1))) ) )

        p_y_ub  = gamma_lb*cp.square( cp.norm( cp.pos(x+y_pred-Pmax*np.ones((N, 1))) ))
        p_y_lb  = gamma_ub*cp.square( cp.norm( cp.pos(Pmin*np.ones((N, 1))-(x+y_pred) ) ))

        recourse_cost = cp.abs(y_pred).T@q +\
                cp.pos(y_pred-r_up).T@q*RecourseReserveCostFactor +\
                cp.pos(y_pred+r_down).T@q*RecourseReserveCostFactor

        Q_val = recourse_cost + p_f_lb + p_f_ub + p_theta_lb + p_theta_ub
        Q_val += p_y_ub + p_y_lb

        Q_vals.append(Q_val)

    constraints_list.append( ones.T@x== ones.T@forecast)
    constraints_list.append( x+r_up <= Pmax*np.ones((N, 1)))
    constraints_list.append( x-r_down >= Pmin*np.ones((N, 1)))
    constraints_list.append( x>=Pmin)
    constraints_list.append( x<=Pmax)
    constraints_list.append( r_up>=0)
    constraints_list.append( r_down>=0)
    constraints_list.append( ones.T@alpha==1)
    constraints_list.append( alpha>=0)

    Q_pred = cp.sum(Q_vals)/M

    da_cost = c.T@x + DayAheadReserveCostFactor*c.T@(r_up+r_down)

    prob = cp.Problem(cp.Minimize( da_cost + Q_pred ), constraints_list)

    prob.solve(verbose = False, solver = cp.ECOS) ## Works!  

    solutions = np.concatenate([x.value.T, r_up.value.T, r_down.value.T], axis=-1)


    return solutions, prob.value, prob.status

In [16]:
def solve_modified_GAP(forecast, num_sce):
    '''
    Implemented by following the GAP model in [Kannan, Luedtke, Roald(2020)].
    I think it is kind of weird that y doesn't have an offset and the second-stage
    is totally determined by the uncertainties.
    So implement modified GAP instead.
    '''

    M = num_sce

    # c = da_cost_Coeff
    # q = linear_cost_Coeff
    B = -Yrr
    F = Arr
    rho_lb = 10.
    rho_ub = 10.
    gamma_lb = 10.
    gamma_ub = 10.
    ones = np.ones((N,1))

    Brr = -Yrr[1:,:]
    Brr_inv = np.linalg.inv(Brr)
    I = np.eye(N)
    R = I[1:,:]
    
    # first-stage decision variables
    x = cp.Variable((N, 1))
    r_up = cp.Variable((N, 1))
    r_down = cp.Variable((N, 1))
    alpha = cp.Variable((N, 1))

    # second-stage decision variables
    beta = cp.Variable((N,1))


    constraints_list = []
    Q_vals = []

    PD = forecast.flatten()
    std_vec = PD*sce_std
    std_vec = np.where(std_vec<1e-5, 0.1, std_vec)

    # generate M uncertainty realizations using the given forecast as mean
    # vector and 5% of it as std.
    scenarios = np.random.multivariate_normal(PD, np.diag(std_vec), M)
    scenarios = np.clip(scenarios, a_min=0., a_max=None).T
    assert scenarios.shape[0] == N

    for m in range(M):
        # for each scenario of noise
        omega =  scenarios[:, m:m+1] - forecast
        net_d = scenarios[:, m:m+1] - x

        y_pred = (ones.T@omega)*alpha + beta
        theta_pred = Brr_inv@R@(net_d-y_pred)

        constraints_list.append( y_pred == Yrr@theta_pred + net_d )

        p_f_lb = rho_lb*cp.square( cp.norm( cp.pos(-Fmax*np.ones((num_lines, 1)) - F@theta_pred) ) )
        p_f_ub = rho_ub*cp.square( cp.norm(  cp.pos(F@theta_pred -Fmax*np.ones((num_lines, 1))) ) )

        p_theta_lb  = rho_lb*cp.square( cp.norm(  cp.pos(-np.pi*np.ones((N-1, 1)) - theta_pred) ) )
        p_theta_ub  = rho_ub*cp.square( cp.norm(  cp.pos(theta_pred - np.pi*np.ones((N-1, 1))) ) )

        p_y_ub  = gamma_lb*cp.square( cp.norm( cp.pos(x+y_pred-Pmax*np.ones((N, 1))) ))
        p_y_lb  = gamma_ub*cp.square( cp.norm( cp.pos(Pmin*np.ones((N, 1))-(x+y_pred) ) ))

        recourse_cost = cp.abs(y_pred).T@q +\
                cp.pos(y_pred-r_up).T@q*RecourseReserveCostFactor +\
                cp.pos(y_pred+r_down).T@q*RecourseReserveCostFactor

        Q_val = recourse_cost + p_f_lb + p_f_ub + p_theta_lb + p_theta_ub
        Q_val += p_y_ub + p_y_lb

        Q_vals.append(Q_val)

    constraints_list.append( ones.T@x== ones.T@forecast)
    constraints_list.append( x+r_up <= Pmax*np.ones((N, 1)))
    constraints_list.append( x-r_down >= Pmin*np.ones((N, 1)))
    constraints_list.append( x>=Pmin)
    constraints_list.append( x<=Pmax)
    constraints_list.append( r_up>=0)
    constraints_list.append( r_down>=0)
    constraints_list.append( ones.T@alpha==1)
    constraints_list.append( alpha>=0)

    Q_pred = cp.sum(Q_vals)/M

    da_cost = c.T@x + DayAheadReserveCostFactor*c.T@(r_up+r_down)

    prob = cp.Problem(cp.Minimize( da_cost + Q_pred ), constraints_list)

    prob.solve(verbose = False, solver = cp.ECOS) ## Works!  

    solutions = np.concatenate([x.value.T, r_up.value.T, r_down.value.T], axis=-1)


    return solutions, prob.value, prob.status

In [17]:
def solve_AP(forecast, num_sce):
    '''
    This solver more flexible, only assume y is affine in total imbalance, and do not
    enforce the coefficients summing up to 1. both alpha and beta are determined after observing 
    uncertainty realizations.
    '''

    M = num_sce

    # c = da_cost_Coeff
    # q = linear_cost_Coeff
    B = -Yrr
    F = Arr
    rho_lb = 10.
    rho_ub = 10.
    gamma_lb = 10.
    gamma_ub = 10.
    ones = np.ones((N,1))

    Brr = -Yrr[1:,:]
    Brr_inv = np.linalg.inv(Brr)
    I = np.eye(N)
    R = I[1:,:]
    
    # first-stage decision variables
    x = cp.Variable((N, 1))
    r_up = cp.Variable((N, 1))
    r_down = cp.Variable((N, 1))
    
    # second-stage decision variables
    alpha = cp.Variable((N, 1))
    beta = cp.Variable((N,1))


    constraints_list = []
    Q_vals = []

    PD = forecast.flatten()
    std_vec = PD*sce_std
    std_vec = np.where(std_vec<1e-5, 0.1, std_vec)

    # generate M uncertainty realizations using the given forecast as mean
    # vector and 5% of it as std.
    scenarios = np.random.multivariate_normal(PD, np.diag(std_vec), M)
    scenarios = np.clip(scenarios, a_min=0., a_max=None).T
    assert scenarios.shape[0] == N

    for m in range(M):
        # for each scenario of noise
        omega =  scenarios[:, m:m+1] - forecast
        net_d = scenarios[:, m:m+1] - x

        y_pred = (ones.T@omega)*alpha + beta
        theta_pred = Brr_inv@R@(net_d-y_pred)

        constraints_list.append( y_pred == Yrr@theta_pred + net_d )

        p_f_lb = rho_lb*cp.square( cp.norm( cp.pos(-Fmax*np.ones((num_lines, 1)) - F@theta_pred) ) )
        p_f_ub = rho_ub*cp.square( cp.norm(  cp.pos(F@theta_pred -Fmax*np.ones((num_lines, 1))) ) )

        p_theta_lb  = rho_lb*cp.square( cp.norm(  cp.pos(-np.pi*np.ones((N-1, 1)) - theta_pred) ) )
        p_theta_ub  = rho_ub*cp.square( cp.norm(  cp.pos(theta_pred - np.pi*np.ones((N-1, 1))) ) )

        p_y_ub  = gamma_lb*cp.square( cp.norm( cp.pos(x+y_pred-Pmax*np.ones((N, 1))) ))
        p_y_lb  = gamma_ub*cp.square( cp.norm( cp.pos(Pmin*np.ones((N, 1))-(x+y_pred) ) ))

        recourse_cost = cp.abs(y_pred).T@q +\
                cp.pos(y_pred-r_up).T@q*RecourseReserveCostFactor +\
                cp.pos(y_pred+r_down).T@q*RecourseReserveCostFactor

        Q_val = recourse_cost + p_f_lb + p_f_ub + p_theta_lb + p_theta_ub
        Q_val += p_y_ub + p_y_lb

        Q_vals.append(Q_val)

    constraints_list.append( ones.T@x== ones.T@forecast)
    constraints_list.append( x+r_up <= Pmax*np.ones((N, 1)))
    constraints_list.append( x-r_down >= Pmin*np.ones((N, 1)))
    constraints_list.append( x>=Pmin)
    constraints_list.append( x<=Pmax)
    constraints_list.append( r_up>=0)
    constraints_list.append( r_down>=0)

    Q_pred = cp.sum(Q_vals)/M

    da_cost = c.T@x + DayAheadReserveCostFactor*c.T@(r_up+r_down)

    prob = cp.Problem(cp.Minimize( da_cost + Q_pred ), constraints_list)

    prob.solve(verbose = False, solver = cp.ECOS) ## Works!  

    solutions = np.concatenate([x.value.T, r_up.value.T, r_down.value.T], axis=-1)


    return solutions, prob.value, prob.status

In [18]:
def solve_SAA(forecast, num_sce):
    '''
    Given a single instance of forecast, solve the SAA version
    of the true problem.
    This is called cp policy in our work.
    '''

    M = num_sce
    ones = np.ones((N,1))

    x = cp.Variable((N, 1))
    r_up = cp.Variable((N, 1))
    r_down = cp.Variable((N, 1))
    theta = cp.Variable((N-1,M))
    y = cp.Variable((N, M))

    constraints_list = []

    PD = forecast.flatten()
    std_vec = PD*sce_std
    std_vec = np.where(std_vec<1e-5, 0.1, std_vec)

    scenarios = np.random.multivariate_normal(PD, np.diag(std_vec), M)
    scenarios = np.clip(scenarios, a_min=0., a_max=None).T
    assert scenarios.shape[0] == N

    for m in range(M):

        net_d = scenarios[:, m:m+1] - x

        constraints_list.append( y[:,m:m+1] == Yrr@theta[:,m:m+1] + net_d )

        constraints_list.append( x+y[:,m:m+1]<=Pmax*np.ones((N, 1)))
        constraints_list.append( x+y[:,m:m+1]>=Pmin*np.ones((N, 1)))
        # constraints_list.append( ones.T@y[:,m:m+1]<=ones.T@r_up)
        # constraints_list.append( ones.T@y[:,m:m+1]>=-ones.T@r_down)
            
    constraints_list.append( Arr@theta <= Fmax*np.ones((num_lines, M)))
    constraints_list.append( Arr@theta >= -Fmax*np.ones((num_lines, M)))

    constraints_list.append( theta<=np.pi)
    constraints_list.append( theta>=-np.pi)
    
    constraints_list.append( ones.T@x== ones.T@forecast)
    constraints_list.append( x+r_up <= Pmax*np.ones((N, 1)))
    constraints_list.append( x-r_down >= Pmin*np.ones((N, 1)))
    constraints_list.append( x>=Pmin)
    constraints_list.append( x<=Pmax)
    constraints_list.append( r_up>=0)
    constraints_list.append( r_down>=0)

    Q_vals = []
    for m in range(M):

        # total_cost += cp.quad_form(y[:,m:m+1], quad_cost_Coeff) + y[:,m:m+1].T@linear_cost_Coeff
        recourse_cost = cp.abs(y[:,m:m+1]).T@q +\
                        cp.pos(y[:,m:m+1]-r_up).T@q*RecourseReserveCostFactor +\
                        cp.pos(y[:,m:m+1]+r_down).T@q*RecourseReserveCostFactor
        Q_vals.append(recourse_cost)

    Q_pred = cp.sum(Q_vals)/M
    da_cost = c.T@x + DayAheadReserveCostFactor*c.T@(r_up+r_down)
    prob = cp.Problem(cp.Minimize( da_cost + Q_pred ), constraints_list)
   
    # prob.solve(verbose = False, solver = cp.ECOS) ## Works! 
    # prob.solve(verbose = False, solver = cp.MOSEK) ## Works!
    prob.solve(verbose = False, solver = cp.SCS) ## Works!
    # if prob.status in ["infeasible", "unbounded"]:
    #     print("Problem is " + str(prob.status))

    solutions = np.concatenate([x.value.T, r_up.value.T, r_down.value.T], axis=-1)


    return solutions, prob.value, prob.status


def solver_outer_loop(forecasts, num_sce, solver_name):
    '''
    Given a batch of instances and solve rld problem for each instance.
    We only care about x for now.
    Can return computation time for each instance at the same time (in minutes)
    '''

    num_points = forecasts.shape[0]

    x_vals = []
    total_cost_vals = []
    times = []


    for it in range(num_points):

        forecast = forecasts[it,:].reshape(-1,1)

        start_time = time.time()

        if solver_name == 'saa':
            x_value, prob_value, prob_status = solve_SAA(forecast, num_sce)

        elif solver_name == 'ap':
            x_value, prob_value, prob_status = solve_AP(forecast, num_sce)

        elif solver_name == 'm_gap':
            x_value, prob_value, prob_status = solve_modified_GAP(forecast, num_sce)

        elif solver_name == 'gap':
            x_value, prob_value, prob_status = solve_GAP(forecast, num_sce)

        # elif solver_name == 'pwa':
        #     x_value, prob_value, prob_status = solve_PWA(forecast, num_sce)

        else:
            print('No such solver exists!')

        if prob_status in ["infeasible", "unbounded"]:
            print("Problem is " + str(prob_status))
            x_value = np.inf*np.ones((N, 1))
            prob_value = np.inf

        end_time = time.time()
        times.append((end_time-start_time)/60)

        x_vals.append(x_value.flatten())
        total_cost_vals.append(prob_value)

        # print("--- %s minutes ---" % ((time.time() - start_time)/60))
        
    x_vals = np.array(x_vals)
    total_cost_vals = np.array(total_cost_vals).reshape(-1,1)
    times = np.array(times)

    return x_vals, total_cost_vals, times


### evaluate policy

In [19]:
def evaluate_Q(forecast, x, num_sce):
    '''
    Given a single instance and the initial dispatch x, and evaluate the quality
    of x.
    This function can be applied to the decision obtained from any policy.
    '''

    M = num_sce 
    ones = np.ones((N,1))

    p0 = x[:N,:]
    r_up = x[N:N+N,:]
    r_down = x[N+N:,:]

    theta = cp.Variable((N-1,M))
    y = cp.Variable((N, M))


    net_d_vals = np.zeros((N, M))

    constraints_list = []

    PD = forecast.flatten()
    std_vec = PD*sce_std
    std_vec = np.where(std_vec<1e-5, 0.1, std_vec)

    scenarios = np.random.multivariate_normal(PD, np.diag(std_vec), M)
    scenarios = np.clip(scenarios, a_min=0., a_max=None).T
    assert scenarios.shape[0] == N

    for m in range(M):
        net_d = scenarios[:, m:m+1] - p0
        
        # net_d_vals[:,m:m+1] = net_d

        constraints_list.append( y[:,m:m+1] == Yrr@theta[:,m:m+1] + net_d )
        constraints_list.append( p0 + y[:,m:m+1]<=Pmax*np.ones((N, 1)))
        constraints_list.append( p0 + y[:,m:m+1]>=Pmin*np.ones((N, 1)))
        # constraints_list.append( ones.T@y[:,m:m+1]<=ones.T@r_up)
        # constraints_list.append( ones.T@y[:,m:m+1]>=-ones.T@r_down)
        
    constraints_list.append( Arr@theta <= Fmax*np.ones((num_lines, M)))
    constraints_list.append( Arr@theta >= -Fmax*np.ones((num_lines, M)))

    constraints_list.append( theta<=np.pi)
    constraints_list.append( theta>=-np.pi)

    Q_vals = []
    for m in range(M):
        recourse_cost = cp.abs(y[:,m:m+1]).T@q+\
                cp.pos(y[:,m:m+1]-r_up).T@q*RecourseReserveCostFactor +\
                cp.pos(y[:,m:m+1]+r_down).T@q*RecourseReserveCostFactor 

        Q_vals.append(recourse_cost)

    Q_pred = cp.sum(Q_vals)/M

    prob = cp.Problem(cp.Minimize( Q_pred ), constraints_list)

    prob.solve(verbose = False, solver = cp.SCS) ## Works!   

    if prob.status in ["infeasible", "unbounded"]:
            return prob.value, np.inf*np.ones((N,1)), prob.status

    return prob.value, np.sum(y.value, axis=1)/M, prob.status


def evaluate_outer_loop(forecasts, x_pred, num_sce):
    '''
    Given a batch of instances and the associated predictions of first stage decision,
    and evaluate these initial dispatch decisions for each instance.

    '''

    num_points = forecasts.shape[0]

    Q_pred = []
    y_pred = []

    for i in range(num_points):
        forecast = forecasts[i,:].reshape(-1,1)
        init_dispatch = x_pred[i,:].reshape(-1,1)

        Q_value, avg_y_value, prob_status = evaluate_Q(forecast, init_dispatch, num_sce)

        if prob_status in ["infeasible", "unbounded"]:
            print("Problem is " + str(prob_status))
            avg_y_value = np.inf*np.ones((N, 1))
            Q_value = np.inf

        Q_pred.append(Q_value)
        y_pred.append(avg_y_value.flatten())
        

    Q_pred = np.array(Q_pred).reshape(-1,1)
    y_pred = np.array(y_pred)

    p0 = x_pred[:,:N]
    r_up = x_pred[:,N:N+N]
    r_down = x_pred[:,N+N:]

    da_cost = (p0+r_up+r_down)@c
    total_cost_pred = (p0+r_up+r_down)@c + Q_pred
    

    return total_cost_pred, Q_pred, y_pred

# load.py

In [20]:
nominal_PD_data = PD_data.reshape(1,-1)

# Load dataset

train_dataset = np.load(data_path+'train_set.npy')
test_dataset = np.load(data_path+'test_set.npy')
pretrain_dataset = np.load(data_path+'pretrain_set.npy')

Ntr = train_dataset.shape[0]
Ntr2 = pretrain_dataset.shape[0]
Ntst = test_dataset.shape[0]


print('train set:', train_dataset.shape)
print('test set:', test_dataset.shape)
print('pretrain set:', pretrain_dataset.shape)

train set: (50000, 39)
test set: (100, 39)
pretrain set: (100, 39)


# benchmarks(gurobi)

In [22]:
!pip install gurobipy>=9.5.1

In [26]:
# Gurobi WLS license file
# Your credentials are private and should not be shared or copied to public repositories.
# Visit https://license.gurobi.com/manager/doc/overview for more information.
WLSACCESSID='cb9b0a86-05f8-4e07-b0c4-6eac1ede9e8c'
WLSSECRET='52f11ad6-e6ea-41ea-b78b-3ebf1876527e'
LicenseID = 866142


In [27]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

In [28]:
# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', WLSACCESSID)
e.setParam('WLSSECRET', WLSSECRET)
e.setParam('LICENSEID', LicenseID)
e.start()

# Create the model within the Gurobi environment
# model = gp.Model(env=e)
solve_saa = gp.Model(env=e,name='solve_saa')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID


GurobiError: ignored

## Input data
Define all the input data for the model.

In [40]:
# Pmax_vec = Pmax*np.ones((N,1))
# Pmin_vec = Pmin*np.ones((N,1))
# Fmax_vec = Fmax*np.ones((L,1))
forecast = pretrain_dataset[:1,:]
ones = np.ones((N,1))
c_r = DayAheadReserveCostFactor*c

M = 10

# generate M uncertainty realizations
PD = forecast.flatten()
std_vec = PD*sce_std
std_vec = np.where(std_vec<1e-5, 0.1, std_vec)

scenarios = np.random.multivariate_normal(PD, np.diag(std_vec), M)
scenarios = np.clip(scenarios, a_min=0., a_max=None).T
assert scenarios.shape[0] == N


## Model deployment

In [43]:
# add variables x, r_up, r_down indexed by {1,2,...,N}
# add variables y, theta: y indexed by {1,2,...,N} times {1,2,...,M}
# and theta indexed by {1,2,...,N-1} times {1,2,...,M}
# where M is the number of uncertainty realizations (named scenarios here)

# Define sets and indices
GenSet = list(range(N))
LineSet = list(range(L))
SceSet = list(range(M))

# Add variables
x = solve_saa.addVars(GenSet, lb=Pmin, ub=Pmax, 
                      name="p0")
# Add non-negative variables r_up, r_down
r_up = solve_saa.addVars(GenSet, name="up reserve")
r_down = solve_saa.addVars(GenSet,  name="down reserve")
# Add matrix variables y and theta
y = solve_saa.addVars(GenSet, SceSet, name="recourse")
theta = solve_saa.addVars(LineSet, SceSet, name="angles")

# Add constraints

solve_saa.addConstr(x.sum('*') == np.sum(forecast), 'initial balance')
solve_saa.addConstrs( (x[i] + r_up[i] <= Pmax for i in GenSet), 'up reserve')
solve_saa.addConstrs( (x[i] - r_down[i] >= Pmin for i in GenSet), 'down reserve')


# Set objective function
solve_saa.setObjective(gp.quicksum(c[i,0]*x[i] + c_r[i,0]*(r_up[i]+r_down[i]) for i in GenSet))

# Verify model formulation
solve_saa.write('solve_saa.lp')

# Run optimization engine
solve_saa.optimize()



Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads


GurobiError: ignored